#### TransMonEE Indicators - API (Helix and UIS) sources populated in Data Dictionary
In this notebook, I will loop along these indicators for extraction and transformation.

**Numbers:**
* Helix sources (28 indicators - 1 missing in dataflow)
* UIS sources (129 indicators)

#### Imports

In [1]:
from data_dictionary import query_dataDict
from fileUtils import fileDownload
from sdmx import sdmx_struc
from extraction.wrap_api_address import wrap_api_address
from transformation.destination import destination
from transformation.dataflow import dataflow
import os
import re
import pandas as pd
import numpy as np

#### TransMonEE countries list - Country ISO codes
##### Countries list is taken from dataflow TransMonEE in UNICEF Warehouse (requested by Eduard)

In [2]:
# UNICEF’s REST API data endpoint for TransMonEE Dataflow
url_endpoint = 'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/ECARO,TRANSMONEE,1.0/'

In [3]:
# address and parameters for dataflow structure request
api_address = url_endpoint + 'all'
api_params = {'format':'sdmx-json', 'detail':'structureOnly'}
# API dataflow structure request
d_flow_struc = fileDownload.api_request(api_address,api_params)

##### Country ISO codes (2 and 3 letters)

In [4]:
# TransMonEE three-letters country codes are taken from its dataflow
country_codes_3 = sdmx_struc.get_all_country_codes(d_flow_struc.json())

In [5]:
# country codes equivalence from excel file in repo root
country_codes_file = "all_countrynames_list.xlsx"
country_codes_df = pd.read_excel(country_codes_file)

In [6]:
# map TMEE country_codes (three-letters/two-letters equivalence)
country_codes_2 = [country_codes_df.CountryIso2[country_codes_df.CountryIso3 == elem].values
                   for elem in country_codes_3.values()]
country_codes_2 = np.unique(np.concatenate(country_codes_2))

In [7]:
# country codes mapping dictionary (three-letters/two-letters)
country_map = {k:v for k,v in zip(country_codes_3.values(),country_codes_2)}
# write dictionary in py file to use it during transformations
path_file = "./transformation/country_map.py"
f = open(path_file, 'w')
f.write('country_map = ' + repr(country_map) + '\n')
f.close()

#### TransMonEE UIS API Key

In [8]:
uis_key = "9d48382df9ad408ca538352a4186791b"

#### Read and Query Data Dictionary

In [9]:
# path to excel data dictionary in repo
data_dict_file = './data_dictionary/indicator_dictionary_TM_v2.xlsx'

In [10]:
# get indicators that are extracted by API (code, address and more in pandas dataframe)
api_code_addr_df = query_dataDict.get_API_code_address_etc(data_dict_file)

#### Extract and Transform Indicators from dataframe `api_code_addr_df`
##### API Extraction: download raw data

In [11]:
# raw data destination path
raw_path = './data_raw/'

##### API Extraction: parameters

In [12]:
# parameters: API request dataflow from Helix
helix_api_params = {'startPeriod':'1950', 'endPeriod':'2050', 'locale':'en'}
# parameters: API request dataflow form UIS
uis_api_params = {**helix_api_params, 'subscription-key':uis_key}

##### API Extraction: headers

In [13]:
# API headers (desired format and compress response)
api_headers = {'Accept':'application/vnd.sdmx.data+csv;version=1.0.0', 'Accept-Encoding':'gzip'}

##### Transformation: map raw data into dataflow TransMonEE in UNICEF Warehouse

In [14]:
# transformed data destination path
trans_path = './data_transformed/'
# name of dataflow TransMonEE in UNICEF warehouse
dataflow_out = "ECARO:TRANSMONEE(1.0)"

In [15]:
# TMEE DSD (data structure definition)
dest_dsd = destination('TMEE')

##### Loop on dataframe `api_code_addr_df`

In [16]:
# actual loop (EXTRACT AND TRANSFORM)
for index, row in api_code_addr_df.iterrows():
    
    # sanity check on strings: strip leading and ending spaces
    url_endpoint = row['Address'].strip()
    indicator_code = row['Code'].strip()
    indicator_source = row['Data_Source'].strip()
    # get source_key from indicator_source
    pattern = "(.*?):"
    source_key = re.findall(pattern, indicator_source)[0].strip()
    indicator_notes = row['Obs_Footnote']
    
    print(f"Dealing with indicator: {indicator_code}")
        
    # wrap API addresses
    api_address = wrap_api_address(source_key, url_endpoint, indicator_code, country_codes_3, country_codes_df)
    
    # wrap API parameters
    if source_key.lower() == 'helix':
        api_params = helix_api_params
    else:
        api_params = uis_api_params
        
    # Skip extraction if indicator already downloaded
    flag_download = os.path.exists(f"{raw_path}{indicator_code}.csv")
    # This skip would need extra info to be executed for update purposes!
    # File names could include the year of execution?
    if flag_download:
        print(f"Indicator {indicator_code} skipped (already downloaded)")
    else:
        # request indicator raw data
        indicator_raw = fileDownload.api_request(api_address,api_params,api_headers)
        # if requests satisfactory
        if indicator_raw.status_code == 200:
            # write raw data to raw file
            raw_file = f"{raw_path}{indicator_code}.csv"
            with open(raw_file, 'wb') as f:
                f.write(indicator_raw.content)
            print(f"Indicator {indicator_code} succesfully downloaded")
    
    # Transform raw_data if it hasn't occured before
    flag_transform = os.path.exists(f"{trans_path}{indicator_code}.csv")
    
    if flag_transform:
        print(f"Transformation for {indicator_code} skipped (already done)")
    elif flag_download:        
        # build dataframe with indicator raw data
        data_raw = pd.read_csv(f"{raw_path}{indicator_code}.csv", dtype=str)

        # retain only codes from csv headers
        raw_columns = data_raw.columns.values
        rename_dict = {k:v.split(':')[0] for k,v in zip(raw_columns,raw_columns)}
        data_raw.rename(columns=rename_dict,inplace=True)

        # get dataflow from data raw anchor [0,0]
        text = data_raw.iloc[0,0]
        pattern = ':(.+?)\('
        dataflow_key = re.findall(pattern, text)[0]

        print(f"Transform indicator: {indicator_code}, from dataflow: {dataflow_key}")

        # instantiate dataflow class with the actual one
        dflow_actual = dataflow(dataflow_key)
        if dflow_actual.cod_map:
            # map the codes - normalization - works 'inplace'
            dflow_actual.map_codes(data_raw)

        # "metadata" from data dictionary: dataflow constants
        # any of these below won't be used if they are dataflow columns
        # Development NOTE: data dictionary info may be overwriten after
        constants = {
            'UNICEF_INDICATOR': indicator_code,
            'DATA_SOURCE': indicator_source,
            'OBS_FOOTNOTE': indicator_notes
        }

        # map the columns
        data_map = dflow_actual.map_dataframe(data_raw, constants)

        # save transformed indicator info independently (through pandas)
        data_trans = pd.DataFrame(columns=dest_dsd.get_csv_columns(), dtype=str)
        data_trans = data_trans.append(data_map)
        # destination Dataflow: corresponding UNICEF Warehouse DSD name
        data_trans['Dataflow'] = dataflow_out
        # save file
        data_trans.to_csv(f"{trans_path}{indicator_code}.csv",index=False)


Dealing with indicator: DM_BRTS
Indicator DM_BRTS skipped (already downloaded)
Transformation for DM_BRTS skipped (already done)
Dealing with indicator: DM_POP_URBN
Indicator DM_POP_URBN skipped (already downloaded)
Transformation for DM_POP_URBN skipped (already done)
Dealing with indicator: DM_FRATE_TOT
Indicator DM_FRATE_TOT skipped (already downloaded)
Transformation for DM_FRATE_TOT skipped (already done)
Dealing with indicator: MNCH_ABR
HTTP error occurred: 404 Client Error: 404 for url: https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/UNICEF,MNCH,1.0/ALB+ARM+AZE+BLR+BIH+BGR+HRV+CZE+EST+GEO+HUN+KAZ+KGZ+LVA+LTU+MNE+MKD+POL+MDA+ROU+RUS+SRB+SVK+SVN+TJK+TUR+TKM+UKR+UZB.MNCH_ABR........?startPeriod=1950&endPeriod=2050&locale=en
Dealing with indicator: NT_BW_LBW
Indicator NT_BW_LBW skipped (already downloaded)
Transformation for NT_BW_LBW skipped (already done)
Dealing with indicator: DM_LIFE_EXP
Indicator DM_LIFE_EXP skipped (already downloaded)
Transformation for DM_LIFE_EXP 

Indicator EDUNF_STU_L3_PRV skipped (already downloaded)
Transformation for EDUNF_STU_L3_PRV skipped (already done)
Dealing with indicator: EDUNF_PRP_L3
Indicator EDUNF_PRP_L3 skipped (already downloaded)
Transformation for EDUNF_PRP_L3 skipped (already done)
Dealing with indicator: EDUNF_STU_L3_GEN_PUB
Indicator EDUNF_STU_L3_GEN_PUB skipped (already downloaded)
Transformation for EDUNF_STU_L3_GEN_PUB skipped (already done)
Dealing with indicator: EDUNF_STU_L3_GEN_PRV
Indicator EDUNF_STU_L3_GEN_PRV skipped (already downloaded)
Transformation for EDUNF_STU_L3_GEN_PRV skipped (already done)
Dealing with indicator: EDUNF_STU_L3_VOC_PUB
Indicator EDUNF_STU_L3_VOC_PUB skipped (already downloaded)
Transformation for EDUNF_STU_L3_VOC_PUB skipped (already done)
Dealing with indicator: EDUNF_STU_L3_VOC_PRV
Indicator EDUNF_STU_L3_VOC_PRV skipped (already downloaded)
Transformation for EDUNF_STU_L3_VOC_PRV skipped (already done)
Dealing with indicator: EDUNF_GER_L1
Indicator EDUNF_GER_L1 skipped (

Indicator EDUNF_STU_L5T8_PUB skipped (already downloaded)
Transformation for EDUNF_STU_L5T8_PUB skipped (already done)
Dealing with indicator: EDUNF_STU_L5T8_PRV
Indicator EDUNF_STU_L5T8_PRV skipped (already downloaded)
Transformation for EDUNF_STU_L5T8_PRV skipped (already done)
Dealing with indicator: EDUNF_PRP_L5T8
Indicator EDUNF_PRP_L5T8 skipped (already downloaded)
Transformation for EDUNF_PRP_L5T8 skipped (already done)
Dealing with indicator: EDUNF_FEP_L5T8
Indicator EDUNF_FEP_L5T8 skipped (already downloaded)
Transformation for EDUNF_FEP_L5T8 skipped (already done)
Dealing with indicator: EDUNF_L1
Indicator EDUNF_L1 skipped (already downloaded)
Transformation for EDUNF_L1 skipped (already done)
Dealing with indicator: EDUNF_L2
Indicator EDUNF_L2 skipped (already downloaded)
Transformation for EDUNF_L2 skipped (already done)
Dealing with indicator: EDUNF_L3
Indicator EDUNF_L3 skipped (already downloaded)
Transformation for EDUNF_L3 skipped (already done)
Dealing with indicator:

#### Data to Upload - Build only one CSV with all data transformed
Could be done with Linux command `sed` for faster performance.

In [24]:
# all csv files with data transformed
files_trans = [file for file in os.listdir(trans_path) if file.endswith(".csv")]
# pandas concat
dest_dsd_df = pd.concat([pd.read_csv(f"{trans_path}{f}", dtype=str) for f in files_trans])

# save file
load_path = './data_2_load/'
load_file = 'TMEE_2_load'
dest_dsd_df.to_csv(f"{load_path}{load_file}.csv",index=False)
